# A Gregorian/Cassegrain telescope with refractive reimaging

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%cd ..

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import astropy.units as u
from astropy import constants as const
from matplotlib import patches
import poppy as po
import rayopt as ro
from scipy.optimize import fsolve

import scipy as sp
rcdef = plt.rcParams.copy()

In [ ]:
class TwoMirror(object):
    """Represent a two-mirror telescope
    The internal representation is in terms of normalized parameters 
    These can be initialized directly or through RC telescope parameters
    Schroeder p.18
    
    The normalized parameters can either be specified through
    {m beta k}: beta is the back focal length in units of f_1
    {m rho k}: rho is the ratio of R_2/R_1
    """
    def __init__(self, m=None, k=None, R_1=None, D_1=None):
        self.m = m
        self.k = k
        self.R_1 = R_1
        self.D_1 = D_1
        
        if (m is not None) and (k is not None):
            self.rho = m * k / (m - 1.)
            self.beta = k * (m + 1.) - 1.
            
    def print_config(self):
        Fn_1 = np.abs(self.R_1) / 2. / self.D_1
        
        # f = m f_1 = -m R_1 / 2
        f_eff = self.m * np.abs(self.R_1) / 2.
        
        Fn_eff = np.abs(f_eff) / self.D_1
        
        print "m, Transverse magnification of secondary: ", self.m
        print "k, Ratio of ray heights at mirror margins: ", self.k
        print "R_1, Radius of curvature of primary: ", self.R_1
        print "D_1, Diamater of primary: ", self.D_1
        print "Derived:"
        print "rho, Ratio of ROC2 to ROC1: ", self.rho
        print "beta, Back focal length in units of f_1: ", self.beta
        print "F_1, F-number of primary: ", Fn_1
        print "f_eff, effective focal length of system: ", f_eff
        print "F_eff, effective F-number of the system: ", Fn_eff
        
        if (self.k > 0.) and (self.k < 1.):
            if self.m >= 1.:
                print "Cassegrain with convex or flat secondary"
            else:
                print "Cassegrain with concave secondary"

        if (self.k >= 1.) and (self.m < 0.):
            print "Inverse Cassegrain with concave secondary"
            
        if (self.k < 0.) and (self.m < 0.):
            print "Gregorian with concave secondary"

    def conics(self):
        m = self.m
        beta = self.beta
        mp = m + 1.
        mm = m - 1.
        mb = m - beta

        m1_conic = -1 - 2. * (1. + beta) / m ** 2. / mb
        m2_conic = -(mp / mm) ** 2. - 2. * m * mp / mb / mm ** 3.
    
        return m1_conic, m2_conic

    def aplanatic_optical_spec(self):
        spec = {}
        spec['D_1'] = self.D_1
        spec['D_2'] = np.abs(self.k) * self.D_1
        spec['R_1'] = self.R_1
        spec['R_2'] = self.R_1 * self.rho
        spec['primary_sec'] = self.R_1 / 2. * (1. - self.k)
        spec['sec_to_focus'] = -self.m * self.k * self.R_1 / 2.
        (spec['K_1'], spec['K_2']) = self.conics() 

        return spec

    def print_aplanatic_optical_spec(self):
        spec = self.aplanatic_optical_spec()
        print "(D_1, D_2)", spec["D_1"], spec["D_2"]
        print "(R_1, R_2)", spec["R_1"], spec["R_2"]
        print "(K_1, K_2)", spec["K_1"], spec["K_2"]
        print "Distance from primary to secondary: ", spec["primary_sec"]
        print "Distance from secondary to focus: ", spec["sec_to_focus"]

    def input_rc(self, f=None, d=None, b=None, D_1=None, gregorian=False):
        """Initialize a two-mirror system for a Ritchey-Chretien 
        f: effective focal length of the system
        d: distance between primary and secondary
        b: back focal length behind secondary 
        D_1: primary_diameter 
    
        A simple way to get a Gregorian is to invert f; this gives the same 
        telescope dimensions. Note that this does not just flip the secondary
        across the focus of the primary. It also changes the radii of curvature
        so that d and b retain their meaning.

        See:
        https://en.wikipedia.org/wiki/Ritchey%E2%80%93Chr%C3%A9tien_telescope
        
        Example test for HST
        >>> tm = TwoMirror()
        >>> hst_f = 57599. * u.mm
        >>> hst_d = 4906. * u.mm
        >>> hst_b = hst_d + 1500. * u.mm
        >>> hst_D_1 = hst_f / 24.
        >>> tm.input_rc(f=hst_f, d=hst_d, b=hst_b, D_1=hst_D_1)
        >>> tm.print_config()
        m, Transverse magnification of secondary:  10.4347737464
        k, Ratio of ray heights at mirror margins:  0.111217208632
        R_1, Radius of curvature of primary:  -11039.8177095 mm
        D_1, Diamater of primary:  2399.95833333 mm
        Derived:
        rho, Ratio of ROC2 to ROC1:  0.123005218776
        beta, Back focal length in units of f_1:  0.271743617418
        F_1, F-number of primary:  2.30000195339
        f_eff, effective focal length of system:  57599.0 mm
        F_eff, effective F-number of the system:  24.0
        Cassegrain with convex or flat secondary

        # Calculate the standard conics for RC
        >>> m = (hst_f - hst_b) / hst_d
        >>> bd = hst_b / hst_d
        >>> m1_conic = -1. - 2. / m ** 3. * bd
        >>> m2_conic = -1. - 2. / (m - 1) ** 3. * (m * (2. * m - 1) + bd)
        >>> print m1_conic, m2_conic
        -1.0022984776 -1.49685888412

        # Compare with conics here
        >>> print tm.conics()
        (-1.0022984775995987, -1.4968588841156243)
        
        >>> tm.print_aplanatic_optical_spec()
        (D_1, D_2) 2399.95833333 mm 266.916666667 mm
        (R_1, R_2) -11039.8177095 mm -1357.9551926 mm
        (K_1, K_2) -1.0022984776 -1.49685888412
        Distance from primary to secondary:  -4906.0 mm
        Distance from secondary to focus:  6406.0 mm
        
        # testing the Gregorian configuration
        >>> tm.input_rc(f=hst_f, d=hst_d, b=hst_b, D_1=hst_D_1, gregorian=True)
        >>> tm.print_aplanatic_optical_spec()
        (D_1, D_2) 2399.95833333 mm 266.916666667 mm
        (R_1, R_2) -8829.95684712 mm 912.128281406 mm
        (K_1, K_2) -0.998823937744 -0.743973763015
        Distance from primary to secondary:  -4906.0 mm
        Distance from secondary to focus:  6406.0 mm
        """        
        if gregorian:
            f_eff = -f
        else:
            f_eff = f
            
        # f = m f_1 implies that m = - f R_1 / 2  
        self.m = ((f_eff - b) / d).value
        
        # another way of seeing this:
        # rho = R_2 / R_1
        #     = \frac{-2 d b}{f-b-d} \frac{-(f-b)}{2 d f} 
        # self.rho = (b * (f-b) / f / (f - b - d)).value
        # determined in terms of rho and m
        # self.k = self.rho * (self.m - 1.) / self.m
        self.k = b.value / f_eff.value
                
        # determined in terms of k and m
        self.rho = self.m * self.k  / (self.m - 1.)
        self.beta = self.k * (self.m + 1.) - 1
    
        self.R_1 = -2. * f_eff / self.m
        self.D_1 = D_1

import doctest
doctest.testmod()

In [ ]:
def focal_length(roc1, roc2, index, thickness):
    inv = 1 / roc1 - 1 / roc2
    inv += (index - 1) * thickness / index / roc1 / roc2
    inv *= (index - 1)
    return 1 / inv

def bfd(roc1, roc2, index, thickness):
    f = focal_length(roc1, roc2, index, thickness)
    bfd = 1. - (index - 1.) * thickness / index / roc1
    return bfd * f

def func(roc, *data):
    #return bfd(roc, -roc, data[0], data[1]) - data[2]
    return focal_length(roc, -roc, data[0], data[1]) - data[2]
    
target = 0.5
print fsolve(func, target, args=(3.5, 0.1, target))

In [ ]:
# top-level parameters
F_number = 3.
shrink = 2.
radius = 740. / 2. * u.mm / shrink
d = 1400. * u.mm
focus_behind_secondary = 900. * u.mm
relay_focal_length = 50. * u.mm
relay_thickness = 4. * u.mm
object_angle = 3.5 * 1.5 / 60.
relay_index = 3.5

#F_number = 10.
#radius = 3.
#d = 5
#focus_behind_secondary = 0.
#relay_focal_length = 0.5
#relay_thickness = 0.03

tm = TwoMirror()
tm.input_rc(f=radius * 2. * F_number, 
            d=d, b=(d - focus_behind_secondary), 
            D_1=radius * 2., gregorian=True)

tm.print_config()
optical_spec = tm.aplanatic_optical_spec()
tm.print_aplanatic_optical_spec()

In [ ]:
s = ro.System(description="Reflective telescope",
          wavelengths=[632.8e-9])
s.object = ro.InfiniteConjugate(angle_deg=object_angle)
s.fields = 0,
start = ro.Spheroid(material=ro.vacuum)
m1 = ro.Spheroid(material=ro.mirror)
m2 = ro.Spheroid(material=ro.mirror)
end = ro.Spheroid()
s.extend([start, m1, m2, end])

m1.distance = d.to(u.mm).value
m1.radius = optical_spec["D_1"].to(u.mm).value / 2.
m2.radius = optical_spec["D_2"].to(u.mm).value / 2.
s.object.pupil.radius = m1.radius
m1.curvature = 1. / optical_spec['R_1'].to(u.mm).value
# rayopt seems to define the curvature along the direction of propagation
# this is reversed at the secondary
m2.curvature = -1. / optical_spec['R_2'].to(u.mm).value
m1.conic = optical_spec['K_1']
m2.conic = optical_spec['K_2']

m1.offset = 0., 0., d.to(u.mm).value
m2.offset = 0., 0., optical_spec['primary_sec'].to(u.mm).value
end.distance = optical_spec['sec_to_focus'].to(u.mm).value

end.curvature = 2*(m1.curvature - m2.curvature)  # field curvature
s.update()

s.update()
s.paraxial.resize()
s.resize_convex()
s.paraxial.refocus()

ro.Analysis(s)

In [ ]:
s = ro.System(description="Relay optic",
          wavelengths=[632.8e-9])

s.object = ro.FiniteConjugate(radius=10.)
#s.object = ro.InfiniteConjugate(angle_deg=1.)
s.fields = 0., 1.
start = ro.Spheroid(material=ro.vacuum)
conic = -1.
l1s = ro.Spheroid(radius=0.2, conic=conic)
l1e = ro.Spheroid(material=ro.vacuum, radius=0.2, conic=conic)
l2s = ro.Spheroid(radius=0.2, conic=conic)
l2e = ro.Spheroid(material=ro.vacuum, radius=0.2, conic=conic)

end = ro.Spheroid()

s.extend([start, l1s, l1e, l2s, l2e, end])

s.object.pupil.radius = 10.

# Relay optics
relay_curvature = 1. / fsolve(func, relay_focal_length.to(u.mm).value,
                              args=(relay_index, relay_thickness.to(u.mm).value,
                                    relay_focal_length.to(u.mm).value))

print "RELAY CURVATURE", relay_curvature
(l1s.curvature, l1e.curvature) = 0., -2. * relay_curvature
(l2s.curvature, l2e.curvature) = 2. * relay_curvature, 0.
#(l1s.curvature, l1e.curvature) = relay_curvature, -relay_curvature
#(l2s.curvature, l2e.curvature) = relay_curvature, -relay_curvature

l1s.distance = relay_focal_length.to(u.mm).value
l1s.material = ro.ModelMaterial(n=relay_index, catalog="basic", solid=True)
l1e.distance = relay_thickness.to(u.mm).value

l2s.distance = 2 * relay_focal_length.to(u.mm).value
l2s.material = ro.ModelMaterial(n=relay_index, catalog="basic", solid=True)
l2e.distance = relay_thickness.to(u.mm).value

# unsure here?; does refocus
end.distance = 200.

end.curvature = 0.  # field curvature

s.update()
s.paraxial.resize()
s.resize_convex()
s.paraxial.refocus()

ro.Analysis(s)

In [ ]:
s = ro.System(description="Telescope with reimaging",
          wavelengths=[632.8e-9])

s.object = ro.InfiniteConjugate(angle_deg=object_angle)
s.fields = 0., 1.
start = ro.Spheroid(material=ro.vacuum)
m1 = ro.Spheroid(material=ro.mirror)
m2 = ro.Spheroid(material=ro.mirror)
conic = -1.
l1s = ro.Spheroid(radius=0.2, conic=conic)
l1e = ro.Spheroid(material=ro.vacuum, radius=0.2, conic=conic)
l2s = ro.Spheroid(radius=0.2, conic=conic)
l2e = ro.Spheroid(material=ro.vacuum, radius=0.2, conic=conic)

end = ro.Spheroid()
s.extend([start, m1, m2, l1s, l1e, l2s, l2e, end])

m1.distance = d.to(u.mm).value
m2.offset = 0, 0, optical_spec['primary_sec'].to(u.mm).value

m1.radius = optical_spec["D_1"].to(u.mm).value / 2.
m2.radius = optical_spec["D_2"].to(u.mm).value / 2.

s.object.pupil.radius = m1.radius

m1.curvature = 1. / optical_spec['R_1'].to(u.mm).value
# rayopt seems to define the curvature along the direction of propagation
# this is reversed at the secondary
m2.curvature = -1. / optical_spec['R_2'].to(u.mm).value

m1.conic = optical_spec['K_1']
m2.conic = optical_spec['K_2']

# Relay optics
relay_curvature = 1. / fsolve(func, relay_focal_length.to(u.mm).value,
                              args=(relay_index, relay_thickness.to(u.mm).value,
                                    relay_focal_length.to(u.mm).value))

print "RELAY CURVATURE", relay_curvature
(l1s.curvature, l1e.curvature) = relay_curvature, -relay_curvature
(l2s.curvature, l2e.curvature) = relay_curvature, -relay_curvature

l1s.distance = optical_spec['sec_to_focus'].to(u.mm).value + relay_focal_length.to(u.mm).value
l1s.material = ro.ModelMaterial(n=relay_index, catalog="basic", solid=True)
l1e.distance = relay_thickness.to(u.mm).value

l2s.distance = 2 * relay_focal_length.to(u.mm).value
l2s.material = ro.ModelMaterial(n=relay_index, catalog="basic", solid=True)
l2e.distance = relay_thickness.to(u.mm).value

# unsure here?; does refocus
end.distance = optical_spec['sec_to_focus'].to(u.mm).value

end.curvature = 0.  # field curvature

s.update()
s.paraxial.resize()
s.resize_convex()
s.paraxial.refocus()

ro.Analysis(s)


In [ ]:
ap = po.CircularAperture(radius=1)
wf = po.FresnelWavefront(beam_radius=1*u.m, wavelength=1. * u.mm)
wf.angular_coordinates = False
wf *= ap
print("After aperture, beam waist is at {}".format(wf.z_w0) )

conv_lens = po.QuadraticLens(10.0*u.m)
wf *= conv_lens
print("After lens with focal length {}, beam waist is at {:.2f}".format(conv_lens.fl, wf.z_w0) )

wf.propagate_fresnel(5. * u.m)
print("After propagating 5 m, the waist is still at {:.2f}".format(wf.z_w0))

plt.figure(figsize=(10,5))
wf.display('both', colorbar=True)
plt.suptitle("Wavefront after propagating 5 m", fontsize=18)

div_lens = po.QuadraticLens(-10*u.m)
wf *= div_lens
print("After lens with focal length {}, beam waist is at {:.2f}".format(div_lens.fl, wf.z_w0) )

#print("Propagating {:.2f} to the final focus".format(wf.z_w0 - 5. *u.m))
#wf.propagate_fresnel(wf.z_w0 - 5. *u.m)
wf.propagate_fresnel(wf.z_w0 - 0.01 *u.m)


plt.figure(figsize=(10,5))
wf.display('both', colorbar=True, imagecrop=None, scale='log', 
           vmax=wf.amplitude.max(), vmin=wf.amplitude.max()*1e-5)
plt.suptitle("Wavefront at focus of lens", fontsize=18);